### Global Validation ###

This notebook combines several validation notebooks: `global_validation_tasmax_v2.ipynb` and `global_validation_dtr_v2.ipynb` along with `check_aiqpd_downscaled_data.ipynb` to create a "master" global validation notebook. It also borrows validation code from the ERA-5 workflow, `validate_era5_hourlyORdaily_files.ipynb`. It is intended to be run with `papermill`. 

### Data Sources ###

Coarse Resolution: 
- CMIP6 
- Bias corrected data 
- ERA-5

Fine Resolution: 
- Bias corrected data 
- Downscaled data 
- ERA-5 (fine resolution)
- ERA-5 (coarse resolution resampled to fine resolution) 

### Types of Validation ### 

Basic: 
- maxes, means, mins  
    - CMIP6, bias corrected and downscaled 
    - historical (1995-2014), 2020-2040, 2040-2060, 2060-2080, 2080-2100 
- differences between historical and future time periods for bias corrected and downscaled
- differences between bias corrected and downscaled data 

Variable-specific: 
- GMST
- days over 95 (TO-DO)
- max # of consecutive dry days, highest precip amt over 5-day rolling window (TO-DO) 

In [19]:
%matplotlib inline 
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os 
import gcsfs 
from matplotlib import cm
import warnings 

from validation import *

### Set Validation Parameters ###

In [23]:
# variable options: 'tasmax', 'tasmin', 'dtr', 'pr'
variable = 'tasmax'
# ssp options: 'ssp126', 'ssp245', 'ssp370', 'ssp585'
ssp = 'ssp370'

# data output types for running validation 
cmip6 = True
bias_corrected = True
downscaled = True
# projection time period options: '2020_2040', '2040_2060', '2060_2080', '2080_2100'
projection_time_period = '2080_2100'

# validation plot options
basic_diagnostics = True

# options: 'mean', 'max', 'min'
basic_diag_type = 'mean'

gmst = True
difference_plots = True

# options: 'downscaled_minus_biascorrected' , 'change_from_historical'
diff_type = 'downscaled_minus_biascorrected'

# contains the gcs URLs to zarr locations for each specified dataset
data_dict = {'coarse': {'cmip6': {'historical': 'scratch/biascorrectdownscale-bk6n8/biascorrectdownscale-bk6n8-858077599/out.zarr', 
                                  ssp: 'scratch/biascorrectdownscale-bk6n8/biascorrectdownscale-bk6n8-269778292/out.zarr'}, 
                        'bias_corrected': {'historical': 'az://biascorrected-stage/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/day/tasmax/gr1/v20210920214427.zarr', 
                                                                                      ssp: 'az://biascorrected-stage/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp370/r1i1p1f1/day/tasmax/gr1/v20210920214427.zarr'}, 
                        'ERA-5':'az://scratch/biascorrectdownscale-bk6n8/biascorrectdownscale-bk6n8-131793962/out.zarr'}, 
             'fine': {'bias_corrected': {'historical': 'az://scratch/biascorrectdownscale-bk6n8/biascorrectdownscale-bk6n8-1362934973/regridded.zarr', 
                                         ssp: 'az://scratch/biascorrectdownscale-bk6n8/biascorrectdownscale-bk6n8-377595554/regridded.zarr'}, 
                      'downscaled': {'historical': 'az://downscaled-stage/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/day/tasmax/gr1/v20210920214427.zarr', 
                                     ssp: 'az://downscaled-stage/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp370/r1i1p1f1/day/tasmax/gr1/v20210920214427.zarr'}, 
                      'ERA-5_fine': 'az://scratch/biascorrectdownscale-bk6n8/biascorrectdownscale-bk6n8-491178896/rechunked.zarr', 
                      'ERA-5_coarse': 'az://scratch/biascorrectdownscale-bk6n8/biascorrectdownscale-bk6n8-1213790070/rechunked.zarr'}}


In [ ]:
# we only plot gmst if validation variable is tasmax 
if variable != 'tasmax': 
    gmst = False
    warnings.warn("gmst plotting option changed to False since validation variable is not tasmax")

### other data inputs ### 

In [ ]:
units = {'tasmax': 'K', 'tasmin': 'K', 'dtr': 'K', 'pr': 'mm'}
years = {'hist': {'start_yr': '1995', 'end_yr': '2014'}, 
              '2020_2040': {'start_yr': '2020', 'end_yr': '2040'}, 
              '2040_2060': {'start_yr': '2040', 'end_yr': '2060'}, 
              '2060_2080': {'start_yr': '2060', 'end_yr': '2080'}, 
              '2080_2100': {'start_yr': '2080', 'end_yr': '2100'}}
years_test = {'hist': {'start_yr': '1995', 'end_yr': '2014'}, 
              '2020_2040': {'start_yr': '2020', 'end_yr': '2040'}, 
              '2040_2060': {'start_yr': '2040', 'end_yr': '2060'}}

### Validation ### 

### basic diagnostic plots: means, maxes, mins ### 

In [ ]:
if cmip6 and basic_diagnostics: 
    plot_diagnostic_climo_periods(read_gcs_zarr(data_dict['coarse']['cmip6']['historical']), 
                                  read_gcs_zarr(data_dict['coarse']['cmip6'][ssp]), 
                                  ssp, years, variable, basic_diag_type, 'cmip6', 
                                  units, vmin=280, vmax=320)

In [ ]:
if bias_corrected and basic_diagnostics: 
    plot_diagnostic_climo_periods(read_gcs_zarr(data_dict['coarse']['bias_corrected']['historical']), 
                                  read_gcs_zarr(data_dict['coarse']['bias_corrected'][ssp]), 
                                  ssp, years, variable, basic_diag_type, 'bias_corrected', 
                                  units, vmin=280, vmax=320)

In [ ]:
if downscaled and basic_diagnostics: 
    plot_diagnostic_climo_periods(read_gcs_zarr(data_dict['coarse']['downscaled']['historical']), 
                                  read_gcs_zarr(data_dict['coarse']['downscaled'][ssp]), 
                                  ssp, years, variable, basic_diag_type, 'downscaled', 
                                  units, vmin=280, vmax=320)

### GMST ### 

In [ ]:
if gmst: 
    plot_gmst_diagnostic(read_gcs_zarr(data_dict['coarse']['cmip6']['historical']), 
                         read_gcs_zarr(data_dict['coarse']['cmip6'][ssp]), 
                         read_gcs_zarr(data_dict['coarse']['bias_corrected']['historical']), 
                         read_gcs_zarr(data_dict['coarse']['bias_corrected'][ssp]), 
                         variable=variable, ssp=ssp, ds_hist_downscaled=None, ds_fut_downscaled=None)

### Difference plots: bias corrected and downscaled OR historical/future (bias corrected and downscaled data outputs) ###

In [ ]:
if bias_corrected and difference_plots:
    plot_bias_correction_downscale_differences(read_gcs_zarr(data_dict['fine']['bias_corrected']['historical']), 
                                               read_gcs_zarr(data_dict['fine']['downscaled']['historical']), 
                                               read_gcs_zarr(data_dict['fine']['bias_corrected'][ssp]), 
                                               read_gcs_zarr(data_dict['fine']['downscaled'][ssp]), 
                                               diff_type, 'bias_corrected', variable,
                                                   ssp=ssp, time_period=projection_time_period)

In [ ]:
if downscaled and difference_plots:
    plot_bias_correction_downscale_differences(read_gcs_zarr(data_dict['fine']['bias_corrected']['historical']), 
                                               read_gcs_zarr(data_dict['fine']['downscaled']['historical']), 
                                               read_gcs_zarr(data_dict['fine']['bias_corrected'][ssp]), 
                                               read_gcs_zarr(data_dict['fine']['downscaled'][ssp]), 
                                               diff_type, 'downscaled', variable,
                                                   ssp=ssp, time_period=projection_time_period)

### TO-DO: Days over 95 degrees F/extreme precip metrics ###